In [44]:
import pandas as pd
import requests
import time
import re

from bs4 import BeautifulSoup

In [37]:
import functions as fun

# Obtaining Data

**Note that much of the following code will be commented out to reduce runtime. DataFrames will be saved as .csv files accordingly.**

## Rating Data

This is actually my third attempt at pulling in the movie info and modeling it. Previously, I had used rottentomatoes.com and imdb.com for best and worst movie info, but they both had their problems. I had gotten all the way through modeling through neural networds before moving on the below strategy, which worked out well.

### Sraping metacritic.com

Metacritic data will be used for classification.

In [10]:
goods_titles = []

for i in range(0,20):
    # There are 10 pages to flip through of 100 movies each.
    page = requests.get(
        'https://www.metacritic.com/browse/movies/score/metascore/all/filtered?page={}'.format(i),
        headers={'User-Agent': 'Chrome/80.0.3987.116'})
    soup = BeautifulSoup(page.content, 'html.parser')
    
    # Now that we've gotten the content from the page, we need to loop through each element.
    for i in range(0,100,1):
        title = soup.find_all('span', class_="title numbered")[i]\
            .next_sibling.next_sibling.contents[1].contents[0]
        goods_titles.append(title)
    
    # We're only pinging 10 times but might as well be safe since it costs like
    # nothing.
    time.sleep(1)

In [11]:
len(goods_titles)

2000

In [12]:
goods_titles[-1]

'Calibre'

In [13]:
goods_titles[:5]

['Citizen Kane', 'The Godfather', 'Rear Window', 'Casablanca', 'Boyhood']

In [14]:
bads_titles = []

for i in range(110, 130):
    # There are 10 pages to flip through of 100 movies each.
    page = requests.get(
        'https://www.metacritic.com/browse/movies/score/metascore/all/filtered?page={}'.format(i),
        headers={'User-Agent': 'Chrome/80.0.3987.116'})
    soup = BeautifulSoup(page.content, 'html.parser')
    
    # Now that we've gotten the content from the page, we need to loop through each element.
    for i in range(0,100,1):
        try:
            title = soup.find_all('span', class_="title numbered")[i]\
            .next_sibling.next_sibling.contents[1].contents[0]
            bads_titles.append(title)
        except:
            pass
    # We're only pinging 10 times but might as well be safe since it costs like
    # nothing.
    time.sleep(1)

In [15]:
len(bads_titles)

2000

In [20]:
goods_formatted = fun.format_titles(goods_titles)
bads_formatted = fun.format_titles(bads_titles)

### Scraping rottentomatoes.com

The rottentomatoes.com information will be used for linear regression.

In [25]:
all_rotten_movies = []
rotten_scores = []
for i in range(0, 101):
    page = requests.get("https://www.rottentomatoes.com/browse/"
                        "dvd-streaming-all?minTomato={}&maxTomato={}&services"
                        "=amazon;hbo_go;itunes;netflix_iw;vudu;amazon_prime;"
                        "fandango_now&genres=1;2;4;5;6;8;9;10;11;13;18;14"
                        "&sortBy=release".format(i, i+1))
    soup = BeautifulSoup(page.content, 'html.parser')
    page = soup.get_text()
    comp = re.compile('"\/m\/\w+"')
    movies = comp.findall(page)
    movies_unique = list(set([movie[4:-1] for movie in movies]))
    rotten_scores.extend([i for _ in movies_unique])
    all_rotten_movies.extend(movies_unique)
    print(i)
    print(movies_unique)
    time.sleep(1)

0
['get_gone', 'breaking_and_exiting', 'the_gallows_act_ii', 'the_murder_of_nicole_brown_simpson', 'black_water_2018', 'mob_town_2019', 'maximum_impact_2018', 'enter_the_anime', '10_minutes_gone', 'dark_crimes', 'the_after_party_2018', 'london_fields', 'air_strike_2018', 'polaroid_2019', 'saving_zoe', 'hallowed_ground_2019', 'imprisoned_2019', 'the_poison_rose', 'drive_2019', 'gotti_2018', 'dead_water', 'the_influence_2019', 'speed_kills', 'solis', 'portals', 'radioflash', 'welcome_to_curiosity', 'john_henry_2020', 'the_child_remains', 'trading_paint', 'killers_anonymous', 'american_dresser']
1
['disaster_movie', 'twisted', 'mortal_kombat_annihilation', 'daddy_day_camp', 'strange_wilderness', 'one_for_the_money', 'meet_the_spartans', 'alone_in_the_dark', 'getaway', 'left_behind_2014', 'kings_ransom', 'epic_movie', 'crossover', 'master_of_disguise', 'baby_geniuses']
2
['mod_squad', 'strange_wilderness', 'getaway', 'epic_movie', 'crossover', 'bless_the_child', 'the_apparition', 'mortal_k

14
['the_adventurers_2017', 'living_among_us', 'the_circle_2017', 'bitter_harvest_2017', 'collateral_beauty', 'damascus_cover', 'sextuplets', 'cook_off', 'transformers_the_last_knight_2017', 'the_hustle_2019', 'vigilante_diaries', 'kill_order', 'correspondence_2016', 'sharknado_the_4th_awakens', 'spark_a_space_tail', 'autumn_lights', 'nine_lives_2016', 'a_score_to_settle', 'regression', 'the_curse_of_sleeping_beauty', 'the_nut_job_2_nutty_by_nature', 'night_hunter_2019', 'the_sea_of_trees', 'the_little_vampire_3d', 'man_down', 'robin_hood_2018', 'the_open_house_2018', 'blood_honey', 'lets_be_evil', 'awake_2019', 'jexi']
15
['pixels', 'kill_me_three_times', 'the_kindness_of_strangers_2019', 'the_darkest_minds', 'the_fanatic_2019', 'sinister_2', 'the_fifth_wave', 'bitter_harvest_2017', 'the_transporter_refueled', 'transformers_the_last_knight_2017', 'alvin_and_the_chipmunks_the_road_chip', 'the_dark_tower_2017', 'dear_dictator_2018', 'the_mummy_2017', 'a_score_to_settle', 'pound_of_flesh

27
['can_you_keep_a_secret', 'little_women_2018', 'axl', 'escape_plan_the_extractors', 'drone_2017', 'the_week_of', 'the_aftermath_2019', 'five_nights_in_maine', 'my_big_fat_greek_wedding_2', 'warcraft', 'traffik_2018', 'sherlock_gnomes', 'the_last_sharknado_its_about_time', 'the_rocky_horror_picture_show_lets_do_the_time_warp_again', 'all_i_see_is_you', 'rambo_last_blood', 'fathers_and_daughters', 'rebel_in_the_rye', 'suicide_squad_2016', 'bright', 'batman_v_superman_dawn_of_justice', 'into_the_ashes', 'sandy_wexler', 'mortal_engines', 'the_wilde_wedding', 'night_school_2018', 'uglydolls', 'london_has_fallen', 'a_million_little_pieces', 'proud_mary', 'suburbicon', 'rattlesnake']
28
['almost_friends', 'sunset_2018', 'new_life_2016', 'the_aftermath_2019', 'iron_sky_the_coming_race', 'five_nights_in_maine', 'my_big_fat_greek_wedding_2', 'warcraft', 'traffik_2018', 'the_rocky_horror_picture_show_lets_do_the_time_warp_again', 'shock_and_awe_2018', 'between_worlds_2018', 'venom_2018', 'fath

40
['revenger', 'the_last_witness_2018', 'maleficent_mistress_of_evil', 'run_the_race', 'dont_let_go_2019', 'only', 'heartlock', 'unplanned', 'tremors_a_cold_day_in_hell', 'hell_fest', 'the_upside', 'something', 'arizona_2018', 'pete_davidson_alive_from_new_york', 'spiral_2018', 'reality_queen', 'blood_paradise', 'gauguin_voyage_to_tahiti', 'the_girl_in_the_spiders_web', 'vita_and_virginia', 'gringo_2018', 'den_of_thieves', 'the_night_of_the_virgin', 'mfkz', 'where_hands_touch', 'the_super', 'under_the_eiffel_tower', 'hell_on_the_border', 'death_house', 'bramptons_own', 'nekrotronic', 'thriller_2018']
41
['the_prodigy_2019', 'katie_says_goodbye', 'vincent_n_roxxy', 'dont_let_go_2019', 'stuber', 'black_butterfly_2017', 'the_professor_and_the_madman', 'hell_fest', 'the_upside', 'a_wrinkle_in_time_2018', 'godzilla_king_of_the_monsters_2019', 'seeds_2019', 'spinning_man', 'the_song_of_sway_lake', 'the_laundromat', 'let_fury_have_the_hour_2013', 'the_valley_2018', 'goon_last_of_the_enforcer

53
['guns_akimbo', 'this_changes_everything_2015', '47_meters_down', 'once_upon_a_deadpool', 'demolition_2016', 'elizabeth_harvest', 'five_feet_apart', 'the_lion_king_2019', 'mowgli_legend_of_the_jungle', 'jack_of_the_red_hearts', 'at_first_light', 'ismaels_ghosts', '3_from_hell', 'risen_2016', 'person_to_person_2017', 'the_strange_ones_2018', 'bad_samaritan', 'the_best_of_enemies_2019', 'destination_wedding', 'the_belko_experiment', 'jason_bourne', 'london_town_2016', 'genius_2016', 'woman_walks_ahead', 'one_more_time_2016', 'book_club', 'blood_fest', 'the_last_laugh_2019', 'roman_j_israel_esq', 'american_dreamer_2019', 'crash_pad', 'triple_9']
54
['pali_road_2016', 'guns_akimbo', 'can_we_take_a_joke_2016', 'speech_and_debate_2017', 'summer_night_2019', '47_meters_down', 'once_upon_a_deadpool', 'five_feet_apart', 'rules_dont_apply', 'jack_of_the_red_hearts', 'the_lodgers', 'the_lego_ninjago_movie', 'extremely_wicked_shockingly_evil_and_vile', 'jt_leroy', 'at_first_light', 'ma_2019', '

66
['nancy_drew_and_the_hidden_staircase_2019', 'the_banana_splits', 'another_wolfcop', 'skin_in_the_game_2019', 'little_joe', 'bleach_2018', 'lou_andreas_salome_the_audacity_to_be_free', 'dark_encounter', 'olympic_dreams', 'knives_and_skin', 'long_time_coming_a_1955_baseball_story', 'fast_and_furious_presents_hobbs_and_shaw', 'the_wave_2020', 'welcome_to_mercy', 'wine_country', 'red_letter_day', 'golden_boy_2018', 'vice_2018', 'the_white_crow', 'on_chesil_beach', 'calling_all_earthlings', 'the_house_with_a_clock_in_its_walls', 'beautifully_broken', 'the_quiet_one_2019', '10x10', 'killer_kate', 'the_isle_2019', 'ibiza', 'caught_2018', 'danger_close_2019', 'body_at_brighton_rock', 'lizzie_2018']
67
['damsel', 'nancy_drew_and_the_hidden_staircase_2019', 'the_banana_splits', 'adopt_a_highway', 'skin_in_the_game_2019', 'puppet_master_the_littlest_reich', 'bleach_2018', 'lou_andreas_salome_the_audacity_to_be_free', 'dark_encounter', 'olympic_dreams', 'knives_and_skin', 'long_time_coming_a_1

79
['buffalo_boys_2019', 'marianne_and_leonard_words_of_love', 'betty_they_say_im_different', 'cold_brook', 'the_maestro', 'kevin_harts_guide_to_black_history', 'fire_in_paradise', 'auggie', 'go_back_to_china', 'luciferina', 'at_eternitys_gate', 'greenlight', 'sprinter', 'pressing_on_the_letterpress_film', 'a_boy_called_sailboat', 'a_fortunate_man', 'amy_schumer_growing', 'anthony_jeselnik_fire_in_the_maternity_ward', 'escape_and_evasion', 'greener_grass', 'extracurricular', 'st_bernard_syndicate', 'juanita_2019', 'alpha_2018', 'rondo_2018', 'code_8', 'the_bouncer', 'halloween_2018', '30_miles_from_nowhere', 'scary_stories', 'we_are_the_heat', 'good_boys_2019']
80
['betty_they_say_im_different', 'cold_brook', 'the_maestro', 'kevin_harts_guide_to_black_history', 'fire_in_paradise', 'auggie', 'daniel_isnt_real', 'instant_family', 'at_eternitys_gate', 'greenlight', 'sprinter', 'overlord_2018', 'pressing_on_the_letterpress_film', 'this_is_not_berlin', 'a_fortunate_man', 'abominable', 'anth

92
['echo_in_the_canyon', 'wild_rose_2019', 'anya_2019', 'bumblebee', 'the_great_buster_a_celebration', 'cam', 'in_fabric', 'the_last_black_man_in_san_francisco', 'stay_human', 'if_the_dancer_dances', 'us_2019', 'fighting_with_my_family', 'the_last_race_2018', 'the_old_man_and_the_gun', 'the_wolfs_call', 'the_favourite_2018', 'diane_2019', 'wildlife_2018', 'a_vigilante', 'between_the_lines', 'system_crasher', 'rolling_thunder_revue_a_bob_dylan_story_by_martin_scorsese', 'netizens', 'sword_of_trust', 'the_grief_of_others', 'giant_little_ones', 'high_flying_bird', 'rockos_modern_life_static_cling', 'miss_americana', 'searching_2018', 'uncut_gems', 'maria_by_callas']
93
['the_dead_center', 'anya_2019', 'sweetheart', 'the_great_buster_a_celebration', 'cam', 'in_fabric', 'the_last_black_man_in_san_francisco', 'if_the_dancer_dances', 'my_days_of_mercy', 'us_2019', 'fighting_with_my_family', 'the_last_race_2018', 'the_old_man_and_the_gun', 'screwball', 'the_favourite_2018', 'the_rape_of_recy_

In [26]:
# Chopping off the year for those movies that have it.
rotten_movies_noyear = [film[:-5] if film[-4:-2] == '20' else film 
        for film in all_rotten_movies]

In [28]:
rotten_form = fun.format_titles(rotten_movies_noyear)

## Scraping in the Screenplays

In [46]:
import importlib
importlib.reload(fun)

<module 'functions' from '/Users/terryollila/flatiron-ds-course/dsc-capstone-project-v2-online-ds-ft-100719/flatirons_capstone_2/functions.py'>

In [55]:
goods_formatted.pop(0)

'citizen-kane'

In [56]:
the_good, good_errors = fun.grab_screenplays(goods_formatted)

godfather-the


ConnectionError: HTTPSConnectionPool(host='www.springfieldspringfield.co.uk', port=443): Max retries exceeded with url: /movie_script.php?movie=godfather-the (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x1581649b0>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))

In [ ]:
len(good_errors)

In [ ]:
the_bad, bad_errors = fun.grab_screenplays(bads_formatted)

In [ ]:
len(bad_errors)

In [ ]:
df_good = pd.DataFrame([the_good]).T
df_bad = pd.DataFrame([the_bad]).T

In [ ]:
rotten_movies, rotten_errors = fun.grab_screenplays(rotten_form)

In [ ]:
rotten_df = pd.DataFrame(columns=['titles',
                                  'titles_formatted',
                                  'rotten_scores',
                                  'scripts'])

In [ ]:
# rotten_df.titles = rotten_movies_noyear

In [ ]:
# rotten_df.titles_formatted = rotten_form
rotten_df.titles = rotten_form
rotten_df.RottenScores = rotten_scores

In [ ]:
rotten_df.scripts = rotten_df.titles_formatted.apply(
    lambda x: rotten_scripts[0][x])

In [ ]:
df_good.to_csv('df_good_obtain.csv')
df_bad.to_csv('df_bad_obtain.csv')
rotten_df.to_csv('rotten_df_obtain.csv')